In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/kass/secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'Boston, MA'
TERM = 'Vegan'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Bos_Vegan.json"
JSON_FILE

'Data/results_in_progress_Bos_Vegan.json'

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [6]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Bos_Vegan.json already exists. Deleting previous file...
[i] Data/results_in_progress_Bos_Vegan.json not found. Saving empty list to new file.
- 0 previous results found.


110

In [7]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/110 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 1000 previous results found.


In [9]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hdBsXpL9uAllcb2QsqzK7g,redwhite-boneless-ramen-boston-3,REDWHITE BONELESS RAMEN,https://s3-media1.fl.yelpcdn.com/bphoto/nr-GE1...,False,https://www.yelp.com/biz/redwhite-boneless-ram...,410,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 42.348815, 'longitude': -71.084731}","[delivery, pickup]",$$,"{'address1': '294 Newbury St', 'address2': Non...",+18572770609,(857) 277-0609,1111.128590
1,qw5gR8vW7mSOK4VROSwdMA,veggie-galaxy-cambridge,Veggie Galaxy,https://s3-media3.fl.yelpcdn.com/bphoto/mQr5hS...,False,https://www.yelp.com/biz/veggie-galaxy-cambrid...,1468,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 42.3636681909004, 'longitude': -7...",[delivery],$$,"{'address1': '450 Massachusetts Ave', 'address...",+16174971513,(617) 497-1513,3021.578646
2,XnQYoHy23BULUooiqHe42Q,lulu-green-boston,Lulu Green,https://s3-media3.fl.yelpcdn.com/bphoto/jUVAsz...,False,https://www.yelp.com/biz/lulu-green-boston?adj...,90,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 42.339588, 'longitude': -71.051715}","[delivery, pickup]",$$,"{'address1': '246 W Broadway', 'address2': Non...",+16174204070,(617) 420-4070,1850.819001
3,kP1b-7BO_VhWk_0tvuA_tw,carmelinas-boston-2,Carmelina's,https://s3-media2.fl.yelpcdn.com/bphoto/rxZBwI...,False,https://www.yelp.com/biz/carmelinas-boston-2?a...,3341,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 42.36388, 'longitude': -71.05415}","[delivery, restaurant_reservation]",$$,"{'address1': '307 Hanover St', 'address2': '',...",+16177420020,(617) 742-0020,2272.117192
4,pB_mOAgN7JuOdBp0HDA4Hg,plnt-burger-boston,PLNT Burger,https://s3-media2.fl.yelpcdn.com/bphoto/bYO2f5...,False,https://www.yelp.com/biz/plnt-burger-boston?ad...,15,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 42.34546037805935, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '348 Harrison Ave', 'address2': N...",+16178822989,(617) 882-2989,743.951912


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,eAePV-av2Nox4_TpLwsPQA,singhs-dhaba-cambridge,Singh's Dhaba,https://s3-media3.fl.yelpcdn.com/bphoto/u3Vp0Q...,False,https://www.yelp.com/biz/singhs-dhaba-cambridg...,47,"[{'alias': 'indpak', 'title': 'Indian'}]",4.5,"{'latitude': 42.36959, 'longitude': -71.11131}","[delivery, pickup]",NaN,"{'address1': '1001 Massachusetts Ave', 'addres...",+16179451525,(617) 945-1525,4080.374374
996,aIeKRFmCK8tYKPGe7RirwQ,natural-food-exchange-stoneham-2,Natural Food Exchange,https://s3-media4.fl.yelpcdn.com/bphoto/1j9txn...,False,https://www.yelp.com/biz/natural-food-exchange...,35,"[{'alias': 'healthmarkets', 'title': 'Health M...",4.0,"{'latitude': 42.4813566, 'longitude': -71.0951...",[],$$$,"{'address1': '39 Pleasant St', 'address2': 'Un...",+17819448498,(781) 944-8498,14975.661902
997,jISD9TeLpnWOwVxAUafaJA,yard-house-dedham-4,Yard House,https://s3-media3.fl.yelpcdn.com/bphoto/wAo5VT...,False,https://www.yelp.com/biz/yard-house-dedham-4?a...,1046,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",3.5,"{'latitude': 42.23044, 'longitude': -71.1759}","[delivery, pickup]",$$,"{'address1': '200 Legacy Pl', 'address2': '', ...",+17813264644,(781) 326-4644,15637.451277
998,D37JiBDWxz130VOHIjQhxA,bogies-place-boston,Bogie's Place,https://s3-media2.fl.yelpcdn.com/bphoto/aT2UwP...,False,https://www.yelp.com/biz/bogies-place-boston?a...,117,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 42.35517, 'longitude': -71.06229}",[delivery],$$$,"{'address1': '25 Temple Pl', 'address2': None,...",+16173385333,(617) 338-5333,1105.961705
999,fqZYEqAScS3Zuaz9NTpbtA,forcella-boston,Forcella,https://s3-media2.fl.yelpcdn.com/bphoto/pALeVQ...,False,https://www.yelp.com/biz/forcella-boston?adjus...,77,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 42.36352, 'longitude': -71.05386}","[delivery, pickup]",NaN,"{'address1': '33 N Sq', 'address2': None, 'add...",+16179364274,(617) 936-4274,2251.811943


In [10]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

1

In [11]:
# Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [12]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)